In [1]:
import pandas as pd
import instance_generation

In [2]:
filepath = "./../../Data/parc_features/parc_train_features.tsv"

In [3]:
token_df = pd.read_csv(filepath, delimiter='\t', index_col=0)
token_df = token_df[:100]

C:\Users\quiri\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
instance_output = instance_generation.collect_instances_main(token_df)

In [5]:
def seperate_instance_gold(instance_output):
    
    gold_label_list = list()
    pair_list = list()
    for instance_list in instance_output:
        gold_label_list.append(instance_list[2])
        pair_tuple = (instance_list[0], instance_list[1])
        pair_list.append(pair_tuple)
    
    return pair_list, gold_label_list

def create_instance_list(list_of_tuples,df):
    main_attribution = list()
    main_gap = list()

    # Loop through instances meaning content and source span indices
    for instance in pair_list:
        # initiate instance_list and index_list
        attribution_indices = list()

        source, content = instance
        b_source, e_source = source
        b_content, e_content = content

        # For index of source
        for index in range(b_source, e_source+1):  
            attribution_indices.append(index)
        # for index of content
        for index in range(b_content, e_content+1):
            attribution_indices.append(index)
            
        # Find gap indices
        if b_source < b_content: 
            # If content follows source, gap is between last token of source and first of content
            gap_indices = [e_source, b_content]
        else:
            # If source follows content, then visa versa
            gap_indices = [e_content, b_source]

        main_attribution.append(attribution_indices)
        main_gap.append(gap_indices)
        
    return main_attribution, main_gap

In [7]:
# Extract relevant lists
pair_list, gold_labels = seperate_instance_gold(instance_output)

sequence_indices, gap_indices = create_instance_list(pair_list, token_df)

In [8]:
pair_df = pd.DataFrame()
pair_df['source_content_boundaries'] = pair_list
pair_df['gold_labels'] = gold_labels
pair_df['all_indices_in_span'] = sequence_indices
pair_df['gap_indices'] = gap_indices

In [9]:
pair_df.head()

,source_content_boundaries,gold_labels,all_indices_in_span,gap_indices
0,"((58, 59), (57, 89))",0,"[58, 59, 57, 58, 59, 60, 61, 62, 63, 64, 65, 6...","[89, 58]"
1,"((60, 61), (57, 89))",0,"[60, 61, 57, 58, 59, 60, 61, 62, 63, 64, 65, 6...","[89, 60]"
2,"((65, 68), (57, 89))",0,"[65, 66, 67, 68, 57, 58, 59, 60, 61, 62, 63, 6...","[89, 65]"
3,"((72, 73), (57, 89))",0,"[72, 73, 57, 58, 59, 60, 61, 62, 63, 64, 65, 6...","[89, 72]"
4,"((74, 76), (57, 89))",0,"[74, 75, 76, 57, 58, 59, 60, 61, 62, 63, 64, 6...","[89, 74]"


In [13]:
def get_word_length(df, list_of_tuples): 
    """
    Get word length for the content or source in the tuple (specified by s or c)
    """
    
    content_len_list = []
    source_len_list = []
    
    for s_tuple, c_tuple in list_of_tuples:
        # Get content len
        start, end = c_tuple
        content_len_list.append(end+1-start)
            
        # Get source len
        start, end = s_tuple
        source_len_list.append(end+1-start)
    pair_df['content_length'] = content_len_list    
    pair_df['source_length'] = source_len_list
    
def check_s_in_c(token_df, pair_df, list_of_tuples):
    results = []
    for source_indices, content_indices in list_of_tuples:
        b_content, e_content = content_indices
        b_source, e_source = source_indices
        
        s_span = list(token_df.iloc[b_source:e_source+1]["lemma"])
        c_span = list(token_df.iloc[b_content:e_content+1]["lemma"])
        bool_ = all(elem in s_span for elem in c_span)
        
        if bool_ == False:
            result = 0
        else:
            result = 1
        results.append(result)
        
    pair_df['source_in_content'] = results
        
def get_distance_c2sentstart(token_df, pair_df, list_of_tuples): 
    distances_2docstart = []
    distances_2sentstart = []
    
    for source_indices, content_indices in list_of_tuples:
        b_content, e_content = content_indices
        
        # Get the distance from the start of the document to the content span.
        doc_filename = token_df.iloc[b_content]['filename']
        index_doc_start = token_df[token_df.filename == doc_filename].first_valid_index()
        
        distance_c2doc = b_content - index_doc_start
        distances_2docstart.append(distance_c2doc)
        
        
        # Get the distance from the start of the sentence to the content span.
        file_df = token_df.loc[df["filename"] == doc_filename] # Filter by filename to get the correct indices.
        
        sent_id = token_df.iloc[b_content]["sentence_number"]
        index_sent_start = file_df[token_df.sentence_number == sent_id].first_valid_index()
        
        distance_c2sent = b_content - index_sent_start
        distances_2sentstart.append(distance_c2sent)
            
    pair_df['distance_c2docstart'] = distances_2docstart
    pair_df['distance_c2sentstart'] = distances_2sentstart
    


In [14]:
get_word_length(pair_df, pair_df['source_content_boundaries'])
get_distance_c2sentstart(token_df, pair_df, pair_df['source_content_boundaries'])
check_s_in_c(token_df, pair_df, pair_df['source_content_boundaries'])

NameError: name 'df' is not defined

In [12]:
pair_df.head()

NameError: name 'df_pair' is not defined